# Randomly assign mutations to samples in each cancer type

Based on the combination of risk factors, there are 27 possible samples per cancer type. For each of these, we will create 10 different random samples, each containing a fixed set of drivers but random set of passengers. 

We will export two tables (one for skin and another for lung cancers), containing 270 rows each. 

In [1]:
import pandas as pd
import numpy as np

#### Output files 

In [2]:
skin_output_f = 'skin_db.txt'
lung_output_f = 'lung_db.txt'

#### Samples

In [3]:
# Age
var_items1 = ['1a', '1b', '1c']
# Solar exposure / smoking
var_items2 = ['2a', '2b', '2c']
# Solar protection / passive smoking
var_items3 = ['3a', '3b', '3c']
# Sample number
var_items4 = [f'4{i}' for i in 'abcdefghij']
var_items4

['4a', '4b', '4c', '4d', '4e', '4f', '4g', '4h', '4i', '4j']

#### Passenger mutations (common between cancer types)

In [4]:
mutations_f = '/home/claudia/Claudia/outreach/cancerdetective/mutations_db.tsv'
mutations_df = pd.read_csv(mutations_f, sep='\t', header=0)
mutations_df.head(2)

,cancer_type,mutation_id,gene,description,aa_change,dna_change,driver_passenger,og_tsg,targeted_therapy,targeted_therapy_approved
0,lung,FBXW7_R505P,FBXW7,F-box and WD repeat domain containing 7,R505P,chr4:152326136_C>G,driver,tsg,None,False
1,lung,FAM135B_G186E,FAM135B,Family with sequence similarity 135 member B,G186E,chr8:138243054_C>T,driver,og,None,False


In [5]:
passengers_df = mutations_df.loc[mutations_df['driver_passenger'] == 'passenger']
print(len(passengers_df))
passengers_df.head(2)

193


,cancer_type,mutation_id,gene,description,aa_change,dna_change,driver_passenger,og_tsg,targeted_therapy,targeted_therapy_approved
528,lung,OR8B2_A119E,OR8B2,Olfactory receptor family 8 subfamily B member 2,A119E,chr11:124382988_G>T,passenger,unknown,None,False
529,lung,OR51A4_R124G,OR51A4,Olfactory receptor family 51 subfamily A member 4,R124G,chr11:4946731_T>C,passenger,unknown,None,False


In [6]:
passenger_list = passengers_df['mutation_id'].tolist()
len(passenger_list)

193

## Skin melanomas

#### Drivers

Genes: BRAF, NRAS, HRAS, KRAS, CDKN2A, NF1, TP53

Driver mutations are assigned based on TGCA-SKCM frequencies. This is a simplified version: 
- 50% of melanomas are BRAF mutant. 
- 30% of melanomas are Ras mutant: NRAS (most frequently), HRAS or KRAS. These three are mutually exclusive
- 10-15% of melanomas are CDKN2A, or NF1, or TP53 mutated
- BRAF mutations can occur alone or together with CDKN2A, NF1, TP53 
- Ras mutations can occur alone or together with CDKN2A, NF1, TP53

In [7]:
drivers_skin_dict = {
    'a': ['BRAF_V600E'], 
    'b': ['BRAF_G469A'], 
    'c': ['BRAF_V600E', 'TP53_R248G', 'NF1_R1276P'], 
    'd': ['BRAF_V600E', 'CDKN2A_P81L'], 
    'e': ['BRAF_V600E', 'NF1_R1276P'], 
    'f': ['NRAS_Q61H'], 
    'g': ['NRAS_Q61H', 'CDKN2A_P81L'], 
    'h': ['NRAS_Q61K', 'TP53_Y220C', 'PTEN_D162G'], 
    'i': ['KRAS_G12D', 'CDKN2A_P81L'], 
    'j': ['HRAS_G12V', 'TP53_L145R'],
}

In [8]:
lines = []
ctype = 'a'
variable_mutation_values = {'a': 1, 'b':2, 'c':3}
for v1 in var_items1: 
    for v2 in var_items2: 
        for v3 in var_items3: 
            # Sample type 
            sample_type = '_'.join([v1, v2, v3])
            # Number of mutations to be assigned to the sample
            exp_mut_count = sum([variable_mutation_values[i[1]] for i in [v1, v2, v3]])
            
            # Iterate over the 10 random samples in each possible sample type (e.g., 1a_2a_3a)
            for v4 in var_items4: 
                # Get drivers
                sample_drivers = drivers_skin_dict[v4[1]]
                # Get random passengers
                passengers_n = exp_mut_count - len(sample_drivers)
                # Add 1 passenger if none
                passengers_n = 1 if passengers_n == 0 else passengers_n
                sample_passengers = list(np.random.choice(passenger_list, size=passengers_n, replace=False))
                lines.append(pd.DataFrame([[
                    ctype,
                    sample_type, 
                    sample_type + '_' + v4,
                    exp_mut_count,
                    ';'.join(sorted(sample_drivers)), 
                    ';'.join(sorted(sample_passengers)), 
                    ';'.join(sorted(sample_drivers+sample_passengers))
                ]]))
melanomas_results = pd.concat(lines)
melanomas_results.columns = [
    'cancer_type',
    'sample_type', 
    'sample',
    'number_mutations', 
    'drivers', 
    'passengers', 
    'total'
]

In [9]:
melanomas_results.head(15)

,cancer_type,sample_type,sample,number_mutations,drivers,passengers,total
0,a,1a_2a_3a,1a_2a_3a_4a,3,BRAF_V600E,GNAT3_R129Q;OR2T11_S18R,BRAF_V600E;GNAT3_R129Q;OR2T11_S18R
0,a,1a_2a_3a,1a_2a_3a_4b,3,BRAF_G469A,CST9L_G145E;OR5M8_Y131F,BRAF_G469A;CST9L_G145E;OR5M8_Y131F
0,a,1a_2a_3a,1a_2a_3a_4c,3,BRAF_V600E;NF1_R1276P;TP53_R248G,SLC10A2_S335L,BRAF_V600E;NF1_R1276P;SLC10A2_S335L;TP53_R248G
0,a,1a_2a_3a,1a_2a_3a_4d,3,BRAF_V600E;CDKN2A_P81L,SLC17A2_E76K,BRAF_V600E;CDKN2A_P81L;SLC17A2_E76K
0,a,1a_2a_3a,1a_2a_3a_4e,3,BRAF_V600E;NF1_R1276P,NKX6-2_S261L,BRAF_V600E;NF1_R1276P;NKX6-2_S261L
0,a,1a_2a_3a,1a_2a_3a_4f,3,NRAS_Q61H,MOS_A260D;OR5M3_F249L,MOS_A260D;NRAS_Q61H;OR5M3_F249L
0,a,1a_2a_3a,1a_2a_3a_4g,3,CDKN2A_P81L;NRAS_Q61H,OR1L8_P287S,CDKN2A_P81L;NRAS_Q61H;OR1L8_P287S
0,a,1a_2a_3a,1a_2a_3a_4h,3,NRAS_Q61K;PTEN_D162G;TP53_Y220C,EIF4E1B_T218A,EIF4E1B_T218A;NRAS_Q61K;PTEN_D162G;TP53_Y220C
0,a,1a_2a_3a,1a_2a_3a_4i,3,CDKN2A_P81L;KRAS_G12D,OR52B6_S236F,CDKN2A_P81L;KRAS_G12D;OR52B6_S236F
0,a,1a_2a_3a,1a_2a_3a_4j,3,HRAS_G12V;TP53_L145R,LCN1_D90E,HRAS_G12V;LCN1_D90E;TP53_L145R


In [10]:
# Check there are no repeated samples
len(melanomas_results), len(melanomas_results['total'].unique())

(270, 270)

In [11]:
# Export
melanomas_results.to_csv(skin_output_f, sep='\t', header=True, index=False)

## Lung cancers

#### Drivers

Genes: TP53, EGFR, KRAS, NFE2L2, KEAP1, CDKN2A, PTEN, CDH10, KMT2D, KMT2C

Driver mutations are assigned based on TGCA-LUSC and TCGA-LUAD frequencies. This is a simplified version: 
- 60-80% of lung cancers are TP53 mutant
- NFE2L2 and KEAP1 are mutually exclusive

In [12]:
drivers_lung_dict = {
    'a': ['TP53_R248G', 'KRAS_G12C', 'NFE2L2_G31R'], 
    'b': ['TP53_R273H', 'EGFR_L858R'], 
    'c': ['TP53_Y220C', 'EGFR_L861Q'], 
    'd': ['TP53_R248G', 'NFE2L2_G31R'], 
    'e': ['TP53_R273H', 'KEAP1_V155F'], 
    'f': ['EGFR_L861Q'], 
    'g': ['TP53_L145R', 'CDKN2A_P81L'], 
    'h': ['TP53_P151S', 'EGFR_L858R', 'PTEN_D162G'], 
    'i': ['KEAP1_C23Y', 'EGFR_L861Q'], 
    'j': ['EGFR_L858R'],
}

In [13]:
lines = []
ctype = 'b'
variable_mutation_values = {'a': 1, 'b':2, 'c':3}
for v1 in var_items1: 
    for v2 in var_items2: 
        for v3 in var_items3: 
            # Sample type 
            sample_type = '_'.join([v1, v2, v3])
            # Number of mutations to be assigned to the sample
            exp_mut_count = sum([variable_mutation_values[i[1]] for i in [v1, v2, v3]])
            
            # Iterate over the 10 random samples in each possible sample type (e.g., 1a_2a_3a)
            for v4 in var_items4: 
                # Get drivers
                sample_drivers = drivers_lung_dict[v4[1]]
                # Get random passengers
                passengers_n = exp_mut_count - len(sample_drivers)
                # Add 1 passenger if none
                passengers_n = 1 if passengers_n == 0 else passengers_n
                sample_passengers = list(np.random.choice(passenger_list, size=passengers_n, replace=False))
                lines.append(pd.DataFrame([[
                    ctype,
                    sample_type, 
                    sample_type + '_' + v4,
                    exp_mut_count,
                    ';'.join(sorted(sample_drivers)), 
                    ';'.join(sorted(sample_passengers)), 
                    ';'.join(sorted(sample_drivers+sample_passengers))
                ]]))
lung_results = pd.concat(lines)
lung_results.columns = [
    'cancer_type',
    'sample_type', 
    'sample',
    'number_mutations', 
    'drivers', 
    'passengers', 
    'total'
]

In [14]:
lung_results.head(15)

,cancer_type,sample_type,sample,number_mutations,drivers,passengers,total
0,b,1a_2a_3a,1a_2a_3a_4a,3,KRAS_G12C;NFE2L2 G31R;TP53_R248G,RBMY1A1_S111Y,KRAS_G12C;NFE2L2 G31R;RBMY1A1_S111Y;TP53_R248G
0,b,1a_2a_3a,1a_2a_3a_4b,3,EGFR_L858R;TP53_R273H,UGT2B11_L187V,EGFR_L858R;TP53_R273H;UGT2B11_L187V
0,b,1a_2a_3a,1a_2a_3a_4c,3,EGFR_L861Q;TP53_Y220C,OR4S1_R258C,EGFR_L861Q;OR4S1_R258C;TP53_Y220C
0,b,1a_2a_3a,1a_2a_3a_4d,3,NFE2L2_G31R;TP53_R248G,OR2T5_M63I,NFE2L2_G31R;OR2T5_M63I;TP53_R248G
0,b,1a_2a_3a,1a_2a_3a_4e,3,KEAP1_V155F;TP53_R273H,OR5M8_Y131F,KEAP1_V155F;OR5M8_Y131F;TP53_R273H
0,b,1a_2a_3a,1a_2a_3a_4f,3,EGFR_L861Q,KRTAP12-2_T4A;OR52R1_S233T,EGFR_L861Q;KRTAP12-2_T4A;OR52R1_S233T
0,b,1a_2a_3a,1a_2a_3a_4g,3,CDKN2A_P81L;TP53_L145R,OR11A1_P211S,CDKN2A_P81L;OR11A1_P211S;TP53_L145R
0,b,1a_2a_3a,1a_2a_3a_4h,3,EGFR_L858R;PTEN_D162G;TP53_P151S,C8orf86_R125W,C8orf86_R125W;EGFR_L858R;PTEN_D162G;TP53_P151S
0,b,1a_2a_3a,1a_2a_3a_4i,3,EGFR_L861Q;KEAP1_C23Y,OR4K17_L45F,EGFR_L861Q;KEAP1_C23Y;OR4K17_L45F
0,b,1a_2a_3a,1a_2a_3a_4j,3,EGFR_L858R,AMELX_L6I;KCNJ3_R499G,AMELX_L6I;EGFR_L858R;KCNJ3_R499G


In [15]:
# Check there are no repeated samples
len(lung_results), len(lung_results['total'].unique())

(270, 270)

In [16]:
# Export
lung_results.to_csv(lung_output_f, sep='\t', header=True, index=False)